### Line Search Method for Steepest Descent

Overall Goal - to solve minimization problem $: \min_{x \in R^n} f(x), x^* - $ solution.

The most common practice is iteratively look for $x^{k+1}: f(x^{k+1}) < f(x^{k})$, so called descent method.

Eventually, outcome of that search would be "$x^{k+1}$ sufficiently close to $x^*$".

Okay, we have general framework, but how to find that $x^{k+1}$? Previosly I have mentioned that such framework called "descent". So, as descending we can choose the "direction" ($p^k$) and the "length" ($\alpha^k$) of the step $: x^{k+1} := x^k + \alpha^k p^k$.

The most straight-forward choice of search direction is $-\nabla f(x^k)$ as opposite to the direction of greatest growth, is called steepest-descent method. Such choice have some strong advantage like global convergency but in reality numerically it is often not convergent at all. 

Nevertheless, we obtain the following expression $: f(x^k - \alpha^k  \nabla f(x^k)) < f(x^k)$. The last question: which step length $\alpha$ to choose? In old days, guys would solve exact problem to pick good $\alpha^k: \min_{\alpha} f(x^k - \alpha \nabla f(x_k)), \alpha \geq 0$. But this method is not considered cost effective. So, let's just pick randomly some $\alpha$ with hope to get some "good" step length just to demonstrate general method. Later, I will show more optimal startegy to choose $\alpha$.

In [16]:
import warnings
import numpy as np
from typing import Callable
from numpy.typing import NDArray


class SteepestDescent:
    def __init__(self, alpha: float, epsilon: float = 1e-5, max_iter: int = 10**7):
        if alpha < 0 or not isinstance(alpha, (int, float)):
            raise ValueError("'alpha' must be non-negative number.")
        self.alpha = alpha

        if epsilon <= 0 or not isinstance(max_iter, (int, float)):
            raise ValueError("'epsilon' must be small positive number.")
        self._epsilon = epsilon

        if max_iter <= 0 or not isinstance(max_iter, int):
            raise ValueError("'max_iter' must be positive int number.")
        self._max_iter = max_iter

    def solve(self, grad_f: Callable, x0: NDArray) -> NDArray:
        """ 
        The method to find solution of minimizing function 'f' with gradient 'grad_f' and initial guess 'x0'.
        """
        xk = x0.copy()
        for _ in range(0, self._max_iter+1):
            grad_f_xk = grad_f(xk)
            if np.linalg.norm(grad_f_xk) <= self._epsilon:
                break
            xk -= self.alpha * grad_f_xk
        else:
            warnings.warn("You have reached the max iteration limit.")
        return xk
        

f = lambda x: x[0] - x[1] + 2*x[0]*x[1] + 2*x[0]*x[0] + x[1]*x[1]
gf = lambda x: np.array([1+2*x[1]+4*x[0],-1+2*x[0]+2*x[1]])

w_hat = SteepestDescent(alpha=1e-2).solve(gf, np.ones(shape=2))
w_hat, f(w_hat) # expect to obtain: x*=[-1,1.5] and f(x*)=-1.25

(array([-0.99999317,  1.49998895]), np.float64(-1.2499999999355231))

Let's check work of algorithm on more sofisticated case.

$ f(x_1, x_2, x_3) = (x_1+2x_2+x_1x_3)^2 + 2 (x_3 + x_2^2)^2 + c x_1^4$

$\nabla f = \left(\frac{\partial f}{\partial x_1}, \frac{\partial f}{\partial x_2}, \frac{\partial f}{\partial x_3}\right)$

$ \frac{\partial f}{\partial x_1}  = 2(x_1+2x_2+x_1x_3) (x_3 + 1) + 4c x_1^3 $

$ \frac{\partial f}{\partial x_2}  = 4(x_1+2x_2+x_1x_3) + 8 x_2 (x_3 + x_2^2) $

$ \frac{\partial f}{\partial x_3}  = 2 x_1 (x_1+2x_2+x_1x_3) + 4 (x_3 + x_2^2)$

Theoretical solution for different c: $x^* = (0,0,0)$

In [17]:
f_xc = lambda x, c: (x[0]+2*x[1]+x[0]*x[2])*(x[0]+2*x[1]+x[0]*x[2]) + 2 * (x[2]+x[1]*x[1]) * (x[2]+x[1]*x[1]) + c * x[0] * x[0] * x[0] * x[0]
grad_f_xc = lambda x, c: np.array([
    2 * (x[0] + 2*x[1] + x[0]*x[2]) * (x[2] + 1) + 4 * c * x[0] * x[0] * x[0],
    4 * (x[0] + 2*x[1] + x[0]*x[2]) + 8 * x[1] * (x[2] + x[1] * x[1]),
    2 * x[0] * (x[0] + 2*x[1] + x[0]*x[2]) + 4 * (x[2] + x[1] * x[1])
])

from time import perf_counter

t0 = perf_counter()
for c in [1, 10, 100]:
    f_x = lambda x: f_xc(x, c)
    grad_f_x = lambda x: grad_f_xc(x, c)
    x0 = np.ones(shape=3)
    w_hat = SteepestDescent(alpha=1e-2/c).solve(grad_f_x, x0)

    print(f'{c=}')
    print(f'x*={w_hat}')
    print(f'f(x*)={f_x(w_hat)}')
    print(f'f(x*_theory)={f_x([0,0,0])}')
    print(50*"=")
print(f"Time to find solutions: {(perf_counter()-t0)}s")

c=1
x*=[ 1.40862400e-02 -7.04332962e-03 -4.96163716e-05]
f(x*)=3.9372602574777596e-08
f(x*_theory)=0
c=10
x*=[ 6.53829146e-03 -3.26967001e-03 -1.06944045e-05]
f(x*)=1.827623926165688e-08
f(x*_theory)=0
c=100
x*=[-3.03481009e-03  1.51796106e-03 -2.30591004e-06]
f(x*)=8.483795383862314e-09
f(x*_theory)=0
Time to find solutions: 18.460739300004207s


Great. Algorithm works. But, as you can see, we fix different alphas for different cases: in some cases, 0.01 is small enough for fast convergence, but in other cases,  0.01 was too larger (when c=100, gradient became too large -> we need to choose a pretty small alpha value so as not to not jump over solution).

To cope with such scenarios, Wolfe's conditions were proposed, which allow you to choose a specific step length so that it is not too long and not too short.

Wolfe Conditions consist of two conditions:
- Armijo (Sufficient Decrease) Condition: $f(x^k + \alpha p^k)) \leq f(x^k) + c_1 \alpha^k {p^k}^T \nabla f(x^k), p^k = -\nabla f(x^k), c_1 \in (0,1), $ often set as $0.001$. This condition ensures the computed step length can sufficiently decrease the objective function $f(x^k)$.
- Curvature Condition: $\nabla f(x^k + \alpha p^k)^T p^k \geq c_2 \nabla f(x^k)^T p^k, c_2 \in (c_1,1), $ often set as $0.1$. This condition ensures a sufficient increase of the gradient.

Therefore, under these conditions, we can sufficiently well find $\alpha^k$ at each step by narrowing down the range of possible values that satisfy these conditions.

In [18]:
import warnings
import numpy as np
from typing import Callable
from numpy.typing import NDArray


class SteepestDescentWeakWolfeConditions:
    def __init__(self, epsilon: float = 1e-5, max_iter: int = 10**6):
        self.init_alpha = 1.0

        if epsilon <= 0 or not isinstance(max_iter, (int, float)):
            raise ValueError("'epsilon' must be small positive number.")
        self._epsilon = epsilon

        if max_iter <= 0 or not isinstance(max_iter, int):
            raise ValueError("'max_iter' must be positive int number.")
        self._max_iter = max_iter

    def solve(self, f: Callable, grad_f: Callable, x0: NDArray) -> NDArray:
        """ 
        The method to find solution of minimizing function 'f' with gradient 'grad_f' and initial guess 'x0'.
        """
        xk = x0.copy()
        for _ in range(0, self._max_iter+1):
            grad_f_xk = grad_f(xk)
            if np.linalg.norm(grad_f_xk) <= self._epsilon:
                break
            alpha = self._line_search_alpha_under_wolfe_conditions(f, grad_f, xk)
            xk -= alpha * grad_f_xk
        else:
            warnings.warn("You have reached the max iteration limit.")
        return xk
        
    def _line_search_alpha_under_wolfe_conditions(self, f: Callable, grad_f: Callable, x, c1=1e-3, c2=0.1, max_iter=100):
        alpha = self.init_alpha
        alpha_min = 0
        alpha_max = np.inf
        
        for _ in range(max_iter):
            
            # ! Armijo Condition
            if f(x - alpha * grad_f(x)) > f(x) - c1 * alpha * np.dot(-grad_f(x), grad_f(x)):
                alpha_max = alpha
                alpha = (alpha_min + alpha_max) / 2.0
                continue
            
            # Curvature Condition
            if np.dot(grad_f(x - alpha * grad_f(x)), -grad_f(x)) >= c2 * np.dot(grad_f(x), -grad_f(x)):
                return alpha
            
            # -> alpha too small -> need to increase
            
            alpha_min = alpha
            
            if alpha_max == np.inf:
                alpha = alpha * 2.0
            else:
                alpha = (alpha_min + alpha_max) / 2.0
        else:
            warnings.warn("You have reached the max iteration limit for alpha line search.")
        return alpha

f_xc = lambda x, c: (x[0]+2*x[1]+x[0]*x[2])*(x[0]+2*x[1]+x[0]*x[2]) + 2 * (x[2]+x[1]*x[1]) * (x[2]+x[1]*x[1]) + c * x[0] * x[0] * x[0] * x[0]
grad_f_xc = lambda x, c: np.array([
    2 * (x[0] + 2*x[1] + x[0]*x[2]) * (x[2] + 1) + 4 * c * x[0] * x[0] * x[0],
    4 * (x[0] + 2*x[1] + x[0]*x[2]) + 8 * x[1] * (x[2] + x[1] * x[1]),
    2 * x[0] * (x[0] + 2*x[1] + x[0]*x[2]) + 4 * (x[2] + x[1] * x[1])
])

from time import perf_counter

t0 = perf_counter()
for c in [1, 10, 100]:
    f_x = lambda x: f_xc(x, c)
    grad_f_x = lambda x: grad_f_xc(x, c)
    x0 = np.ones(shape=3)
    w_hat = SteepestDescentWeakWolfeConditions().solve(f_x, grad_f_x, x0)

    print(f'{c=}')
    print(f'x*={w_hat}')
    print(f'f(x*)={f_x(w_hat)}')
    print(f'f(x*_theory)={f_x([0,0,0])}')
    print(50*"=")
print(f"Time to find solutions: {(perf_counter()-t0)}s")

c=1
x*=[-1.30705169e-02  6.53475355e-03 -4.27147047e-05]
f(x*)=2.918596741688725e-08
f(x*_theory)=0
c=10
x*=[ 6.00769694e-03 -3.00495991e-03 -9.02957268e-06]
f(x*)=1.3031814943839697e-08
f(x*_theory)=0
c=100
x*=[-2.78638646e-03  1.39432874e-03 -1.94405421e-06]
f(x*)=6.033073327880847e-09
f(x*_theory)=0
Time to find solutions: 0.5274376000161283s


Wooow!!!!! Realy good perfomance boost: 35x speedup under similar results just using wolfe conidtions instead of fixed alpha. Impressive.

The most interesting thing is that this time we solved two optimization problems: external gradient descent and internal optimization of the descent step length parameter. Intuitively, it might seem that the nested optimization problem should, on the contrary, worsen performance, since each time, at each step, a new optimization problem of searching for alpha must be solved. However, if you think about it, these one-dimensional optimizations allowed us to SIGNIFICANTLY reduce the number of external “expensive” multidimensional optimization steps, which is how we achieved the benefit.

However, as you may have noticed, I named the previous class “...WeakWolfeConditions,” and this is not without reason. The fact is that the above conditions can result in an $\alpha$ value that is not close to the minimizer of $\phi(\alpha)$. The (weak) Wolfe conditions can be modified by using the following condition called Strong Wolfe condition, which writes the Curvature condition in absolute values:
$$ p^k \nabla f(x^k + \alpha p^k) \leq c_2 | {p^k}^T f(x^k)|, p^k = - \nabla f(x^k) $$
The Strong Wolfe Curvature condition restricts the slope of $\phi(\alpha)$ from getting too positive, hence excluding points far away from the stationary point of $\phi$. In (weak) Wolfe, if the slope is positive ($>0$), we technically satisfy the condition and stop (or keep going). In Strong Wolfe, a positive slope means we overshot the minimum; therefore, if new slope positive, that alpha must become the new upper bound, forcing the algorithm to search backwards (zoom).

In [19]:
import warnings
import numpy as np
from typing import Callable
from time import perf_counter
from numpy.typing import NDArray


class SteepestDescentStrongWolfeConditions:
    def __init__(self, epsilon: float = 1e-5, max_iter: int = 10**6):
        self.init_alpha = 1.0

        if epsilon <= 0:
            raise ValueError("'epsilon' must be small positive number.")
        self._epsilon = epsilon

        if max_iter <= 0 or not isinstance(max_iter, int):
            raise ValueError("'max_iter' must be positive int number.")
        self._max_iter = max_iter

    def solve(self, f: Callable, grad_f: Callable, x0: NDArray) -> NDArray:
        """ 
        The method to find solution of minimizing function 'f' with gradient 'grad_f' and initial guess 'x0'.
        """
        xk = x0.copy()
        for _ in range(0, self._max_iter+1):
            grad_f_xk = grad_f(xk)
            if np.linalg.norm(grad_f_xk) <= self._epsilon:
                break
            alpha = self._line_search_alpha_under_strong_wolfe_conditions(f, grad_f, xk)
            xk -= alpha * grad_f_xk
        else:
            warnings.warn("You have reached the max iteration limit.")
        return xk
        
    def _line_search_alpha_under_strong_wolfe_conditions(self, f: Callable, grad_f: Callable, x, c1=1e-3, c2=0.1, max_iter=100):
        alpha = self.init_alpha
        alpha_min = 0
        alpha_max = np.inf
        
        for _ in range(max_iter):

            # ! Armijo Condition
            if f(x - alpha * grad_f(x)) > f(x) - c1 * alpha * np.dot(-grad_f(x), grad_f(x)):
                alpha_max = alpha
                alpha = (alpha_min + alpha_max) / 2.0
                continue
            
            # STRONG Wolfe Curvature Condition
            if np.abs(np.dot(grad_f(x - alpha * grad_f(x)), -grad_f(x)) <= c2 * np.abs(np.dot(-grad_f(x), grad_f(x)))):
                return alpha
            
            # If slope is positive, we passed the valley floor -> alpha is new upper bound
            if np.dot(grad_f(x - alpha * grad_f(x)), -grad_f(x)) >= 0:
                alpha_max = alpha
            else:
                # If slope is negative (and steep), we haven't reached valley -> alpha is lower bound
                alpha_min = alpha
            
            if alpha_max == np.inf:
                alpha = alpha * 2.0
            else:
                alpha = (alpha_min + alpha_max) / 2.0
        else:
            warnings.warn("Line search did not converge (Strong Wolfe).")
        return alpha


f_xc = lambda x, c: (x[0]+2*x[1]+x[0]*x[2])*(x[0]+2*x[1]+x[0]*x[2]) + 2 * (x[2]+x[1]*x[1]) * (x[2]+x[1]*x[1]) + c * x[0] * x[0] * x[0] * x[0]
grad_f_xc = lambda x, c: np.array([
    2 * (x[0] + 2*x[1] + x[0]*x[2]) * (x[2] + 1) + 4 * c * x[0] * x[0] * x[0],
    4 * (x[0] + 2*x[1] + x[0]*x[2]) + 8 * x[1] * (x[2] + x[1] * x[1]),
    2 * x[0] * (x[0] + 2*x[1] + x[0]*x[2]) + 4 * (x[2] + x[1] * x[1])
])

from time import perf_counter

t0 = perf_counter()
for c in [1, 10, 100]:
    f_x = lambda x: f_xc(x, c)
    grad_f_x = lambda x: grad_f_xc(x, c)
    x0 = np.ones(shape=3)
    w_hat = SteepestDescentStrongWolfeConditions().solve(f_x, grad_f_x, x0)

    print(f'{c=}')
    print(f'x*={w_hat}')
    print(f'f(x*)={f_x(w_hat)}')
    print(f'f(x*_theory)={f_x([0,0,0])}')
    print(50*"=")
print(f"Time to find solutions: {(perf_counter()-t0)}s")

c=1
x*=[ 1.39420235e-02 -6.97146574e-03 -4.86065655e-05]
f(x*)=3.7786106171351846e-08
f(x*_theory)=0
c=10
x*=[-6.44639588e-03  3.22344759e-03 -1.03951615e-05]
f(x*)=1.7269348598341105e-08
f(x*_theory)=0
c=100
x*=[ 2.98692859e-03 -1.49431634e-03 -2.23393951e-06]
f(x*)=7.962675416221285e-09
f(x*_theory)=0
Time to find solutions: 0.4730938000138849s


So, as we can see, we even got a slight speed boost by slightly changing the alpha search algorithm.

Let's try optimize something even more sophisticated. For example let's maximize likelihood function which is equivalent to minimize cross-entropy ( multiply log-likelihood by $-1/N$) in binary classification problem ($X$ - features, $Y$ - class label, $\hat{Y}$ - predictions by sigmoid).

likelihood function: $ L(w) = \prod_{i=1}^N \hat{y_i}^{y_i}  (1-\hat{y_i})^{1-y_i}$, $w$ - sigmoid weights

cross-entropy loss: $ J(w) = -\frac{1}{N} \sum_{i=1}^N \left( y_i log(\hat{y_i}) + (1-y_i) log(1-\hat{y_i})\right) $

$\nabla J(w) = \frac{1}{N} X^T (\hat{Y} - Y)$

In [52]:
import numpy as np
np.random.seed(666)

N = 50
X = np.random.normal(loc=[-1,3], scale=[1,2**0.5], size=(N,2))
eps = np.random.normal(loc=0,scale=0.1**0.5,size=N)
Y = (3*X[:,0] + X[:,1] + eps > 0).astype(int)


X = np.hstack((X, np.ones((N, 1))))

sigmoid = lambda w: 1 / (1 + np.exp(-X @ w))
cross_entropy = lambda w: -np.mean(Y * np.log(sigmoid(w) + 1e-15) + (1 - Y) * np.log(1 - sigmoid(w) + 1e-15))
grad_cross_entropy = lambda w: 1/N * X.T @ (sigmoid(w) - Y)

w0 = np.ones(shape=3)
w_hat = SteepestDescentStrongWolfeConditions().solve(cross_entropy, grad_cross_entropy, w0)

preds = sigmoid(w_hat) >= 0.5
acc = np.mean(preds == Y)
acc

np.float64(0.96)

Great! 96% accuracy!

We have just solved a classic classification problem using our hand written solver. Good job!

### Conclusion:
In this study, the steepest descent method was implemented and investigated for solving optimization problems.

Experiments showed that using a fixed step size ($\alpha$) is ineffective for functions with complex geometry (using the example of a polynomial with parameter $c=100$), as this leads to slow convergence or the need for manual parameter selection.

The implementation of the linear search algorithm using Wolfe's conditions (weak and strict) allowed the step length to be automatically adapted. This provided a significant increase in performance (up to 35 times faster) and stability of the algorithm, since one-dimensional step optimization significantly reduces the number of expensive gradient calculations in multidimensional space.

The developed optimizer with strict Wolfe conditions was successfully applied to train a logistic regression model (Cross-Entropy minimization).The algorithm correctly found the optimal weights, ensuring 96% classification accuracy.

Thus, the use of adaptive step selection is a critically important component of modern optimization methods.

References:
- https://optimization.cbe.cornell.edu/index.php?title=Line_search_methods#cite_note-2
- https://people.maths.ox.ac.uk/hauser/hauser_lecture2.pdf
- https://indrag49.github.io/Numerical-Optimization/line-search-descent-methods.html